In [ ]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd
import requests

In [ ]:
url = 'https://finance.naver.com/marketindex/?tabSel=exchange#tab_section'

data = requests.get(url)

In [ ]:
data.text

In [ ]:
# 데이터를 파싱 
soup = bs(data.text, 'html.parser')

In [ ]:
soup

In [ ]:
url = 'https://finance.naver.com/marketindex/exchangeList.naver'

In [ ]:
data = requests.get(url)

In [ ]:
soup = bs(data.text, 'html.parser')

In [ ]:
soup

In [ ]:
df = pd.read_html(data.text)[0]

In [ ]:
pd.read_html('https://finance.naver.com/marketindex/exchangeList.naver', encoding='cp949')

In [ ]:
df.head()

In [ ]:
# CSV로 데이터프레임을 저장 
# to_csv(파일명)
df.to_csv('환율.csv', index=False, mode='a', header=False)

In [ ]:
import os

In [ ]:
# 환율 정보 csv 저장 자동화
file_name = "환율.csv"
# 저장하려고 하는 주소에 file_name이 존재하는가?
header_bool = not os.path.exists(file_name)
# file_name not in os.listdir('./')
df = pd.read_html(
        'https://finance.naver.com/marketindex/exchangeList.naver', 
        encoding='cp949'
    )[0]
df.to_csv(
    file_name, 
    mode='a', 
    index = False, 
    header= header_bool
)


In [ ]:
import database

In [ ]:
# sql에 데이터를 대입
mydb = database.MyDB()

In [ ]:
insert_query = """
    insert into `exchange`(
    `통화명`, `매매기준율`, `현찰_사실때`, `현찰_파실때`, `송금_보내실때`, 
    `송금_받으실때`, `미화환산율`)
    values (%s, %s, %s, %s, %s, %s, %s)
"""
mydb.execute_query(insert_query, *df.loc[0].values, inplace=True)

In [ ]:
df.fillna(0, inplace=True)

In [ ]:
# 데이터프레임의 길이만큼 반복 실행하는 반복문을 생성 
for i in range(len(df)):
    # i -> 데이터프레임의 인덱스의 위치 
    mydb.execute_query(insert_query, *df.loc[i].values, inplace=True)

In [ ]:
# !pip install sqlalchemy

In [ ]:
df.columns = df.columns.map(
    lambda x : x[0] if (x[0] == x[1]) else x[0]+'_'+x[1].replace(' ','')
)

In [ ]:
df

In [ ]:
from sqlalchemy import create_engine

In [ ]:
host = '127.0.0.1'
port = 3306
user = 'root'
pwd = '1234'
db = 'ubion'

In [ ]:
# Sqlalchemy에 엔진을 생성 
engine = create_engine(f"mysql+pymysql://{user}:{pwd}@{host}:{port}/{db}")

In [ ]:
df.to_sql(
    name = 'exchange2',  # 테이블 이름
    con = engine,       # 데이터베이스 정보(sqlalchemy engine)
    if_exists = 'append', # 테이블이 존재한다면 추가 옵션
    index = False
)

In [ ]:
# 몽고db에 데이터를 삽입
from pymongo import MongoClient

In [ ]:
client = MongoClient(
    host = '127.0.0.1', 
    port = 27017
)

In [ ]:
db = client['ubion']

In [ ]:
collection = db['exchange']

In [ ]:
# insert_one( dict )
# insert_many( list(dict) )
# 데이터프레임을 list 안에 dict의 형태로 변환
data = df.to_dict(orient='records')

In [ ]:
collection.insert_many(data)

In [ ]:
df = pd.read_html(
        'https://finance.naver.com/marketindex/exchangeList.naver', 
        encoding='cp949'
    )[0]
df.columns = df.columns.map(
    lambda x : x[0] if (x[0] == x[1]) else f"{x[0]}_{x[1].replace(' ', '')}"
)
data = df.to_dict(orient='records')
db['exchange2'].insert_many(data)

In [ ]:
values = list(db['exchange2'].find({}, {'_id' : 0}))

In [ ]:
pd.DataFrame(values)